In [153]:
import os
import pydst

dst = pydst.Dst(lang='da')
n
# make pickle path if not exists
try:
    os.makedirs("pickles")
except:
    pass

In [154]:
#########################
# POPULATION PROJECTION #
#########################

# if a pickle exists, unpickle it
# otherwise get and process the data anew

population_projection = None
if os.path.isfile("pickles/population_projection.pickle"):
    population_projection = pd.read_pickle("pickles/population_projection.pickle")
else:
    raw_data = None
    if os.path.isfile("pickles/raw_data_frkm118.pickle"):
        raw_data = pd.read_pickle("pickles/raw_data_frkm118.pickle")
    else:
        raw_data = dst.get_data(
            table_id = 'FRKM118',
            variables={
                "KØN" : ["*"],
                "ALDER" : ["*"], 
                "TID" :["*"], 
                "OMRÅDE" : ["*"]})
        raw_data.to_pickle("pickles/raw_data_frkm118.pickle")

    
    population_projection = raw_data.copy()
    
    # rename columns
    population_projection = population_projection.rename(columns = {
        "KØN": "gender",
        "ALDER": "age",
        "TID": "year",
        "OMRÅDE": "municipal",
        "INDHOLD": "count"
    })
    
    # map gender to M/F
    population_projection["gender"] = population_projection["gender"].map({
        "Mænd": "M",
        "Kvinder": "F"
    })
    
    # remove total count
    population_projection = population_projection[population_projection["age"] != "Alder i alt"]
    
    # map dst's year to actual int years
    population_projection["age"] = population_projection["age"].map(lambda year : int(year.split(' ', 1)[0]))
    
    population_projection.to_pickle("pickles/population_projection.pickle")

In [155]:
######################
# POPULATION HISTORY #
######################

# do the same thing 👏

population_history = None
if os.path.isfile("pickles/population_history.pickle"):
    population_history = pd.read_pickle("pickles/population_history.pickle")
else:
    raw_data = None
    if os.path.isfile("pickles/raw_data_by2.pickle"):
        raw_data = pd.read_pickle("pickles/raw_data_by2.pickle")
    else:
        raw_data = dst.get_data(
            table_id = "BY2",
            variables= {
                "KØN" : ["*"],
                "ALDER" : ["*"],
                "TID" :["*"],
                "KOMK" : ["*"],
                "BYST" : ["*"]
            })
        
        raw_data.to_pickle("pickles/raw_data_by2.pickle")
        
    population_history = raw_data.copy()
    
    # remove byst column
    population_history = population_history.drop(["BYST"], axis = 1)

    # rename
    population_history = population_history.rename(columns = {
        "KØN": "gender",
        "ALDER": "age",
        "TID": "year",
        "KOMK": "municipal",
        "INDHOLD": "count"
    })

    # map gender to M/F
    population_history["gender"] = population_history["gender"].map({
        "Mænd": "M",
        "Kvinder": "F"
    })

    # map dst's year to actual int years
    population_history["age"] = population_history["age"].map(lambda year : int(year.split(' ', 1)[0]))

    population_history.to_pickle("pickles/population_history.pickle")

In [156]:
population = population_history.append(population_projection)

In [157]:
population.groupby(["municipal", "year", "age"])

ValueError: cannot reindex from a duplicate axis